In [1]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup
import time
import json
import random
import pandas as pd 
import os

In [2]:
def save_data(new_data, filename="amazon_products.json"):
    if os.path.exists(filename):
        with open(filename, "r", encoding="utf-8") as f:
            try:
                old_data = json.load(f)
            except json.JSONDecodeError:
                old_data = []
    else:
        old_data = []
    all_data = old_data + new_data
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(all_data, f, ensure_ascii=False, indent=2)

    print(f"✅ Saved to file: {len(new_data)} new products | Total: {len(all_data)} products.")

#=====================================================================================================

options = uc.ChromeOptions()
options.add_argument("--no-first-run")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = uc.Chrome(options=options)
driver.maximize_window()

#=====================================================================================================

    # Shop by Department categories
url = [
    "https://www.amazon.com/gp/goldbox", # <---> Today's Deals
    # Electronics
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225009011&rh=n%3A%252116225009011%2Cn%3A281407&ref=nav_em__nav_desktop_sa_intl_accessories_and_supplies_0_2_6_2",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225009011&rh=n%3A%252116225009011%2Cn%3A502394&ref=nav_em__nav_desktop_sa_intl_camera_and_photo_0_2_6_3",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225009011&rh=n%3A%252116225009011%2Cn%3A3248684011&ref=nav_em__nav_desktop_sa_intl_car_and_vehicle_electronics_0_2_6_4",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225009011&rh=n%3A%252116225009011%2Cn%3A2811119011&ref=nav_em__nav_desktop_sa_intl_cell_phones_and_accessories_0_2_6_5",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225009011&rh=n%3A%252116225009011%2Cn%3A541966&ref=nav_em__nav_desktop_sa_intl_computers_and_accessories_0_2_6_6",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225009011&rh=n%3A%252116225009011%2Cn%3A172526&ref=nav_em__nav_desktop_sa_intl_gps_and_navigation_0_2_6_7",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225009011&rh=n%3A%252116225009011%2Cn%3A172541&ref=nav_em__nav_desktop_sa_intl_headphones_0_2_6_8",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225009011&rh=n%3A%252116225009011%2Cn%3A667846011&ref=nav_em__nav_desktop_sa_intl_home_audio_0_2_6_9",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225009011&rh=n%3A%252116225009011%2Cn%3A172574&ref=nav_em__nav_desktop_sa_intl_office_electronics_0_2_6_10",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225009011&rh=n%3A%252116225009011%2Cn%3A172623&ref=nav_em__nav_desktop_sa_intl_portable_audio_and_video_0_2_6_11",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225009011&rh=n%3A%252116225009011%2Cn%3A524136&ref=nav_em__nav_desktop_sa_intl_security_and_surveillance_0_2_6_12",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225009011&rh=n%3A%252116225009011%2Cn%3A1266092011&ref=nav_em__nav_desktop_sa_intl_television_and_video_0_2_6_14",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225009011&rh=n%3A%252116225009011%2Cn%3A7926841011&ref=nav_em__nav_desktop_sa_intl_video_game_consoles_and_accessories_0_2_6_15",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225009011&rh=n%3A%252116225009011%2Cn%3A300334&ref=nav_em__nav_desktop_sa_intl_video_projectors_0_2_6_16",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225009011&rh=n%3A%252116225009011%2Cn%3A10048700011&ref=nav_em__nav_desktop_sa_intl_wearable_technology_0_2_6_17",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225009011&rh=n%3A%252116225009011%2Cn%3A2642125011&ref=nav_em__nav_desktop_sa_intl_ebook_readers_and_accessories_0_2_6_18",
    # Computers
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A172456&ref=nav_em__nav_desktop_sa_intl_computer_accessories_and_peripherals_0_2_7_2",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A193870011&ref=nav_em__nav_desktop_sa_intl_computer_components_0_2_7_3",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A13896617011&ref=nav_em__nav_desktop_sa_intl_computers_tablets_0_2_7_4",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A1292110011&ref=nav_em__nav_desktop_sa_intl_data_storage_0_2_7_5",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A3012292011&ref=nav_em__nav_desktop_sa_intl_external_components_0_2_7_6",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A3011391011&ref=nav_em__nav_desktop_sa_intl_laptop_accessories_0_2_7_7",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A1292115011&ref=nav_em__nav_desktop_sa_intl_monitors_0_2_7_8",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A172504&ref=nav_em__nav_desktop_sa_intl_networking_products_0_2_7_9",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A17854127011&ref=nav_em__nav_desktop_sa_intl_power_strips_and_surge_protectors_0_2_7_10",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A172635&ref=nav_em__nav_desktop_sa_intl_printers_0_2_7_11",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A172584&ref=nav_em__nav_desktop_sa_intl_scanners_0_2_7_12",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A11036071&ref=nav_em__nav_desktop_sa_intl_servers_0_2_7_13",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A2348628011&ref=nav_em__nav_desktop_sa_intl_tablet_accessories_0_2_7_14",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A15524379011&ref=nav_em__nav_desktop_sa_intl_tablet_replacement_parts_0_2_7_15",
    "https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A16285851&ref=nav_em__nav_desktop_sa_intl_warranties_and_services_0_2_7_16"
]

#=====================================================================================================
for category in url:
    for attempt in range(3):  
        try:
            driver.set_page_load_timeout(60)  
            driver.get(category)
            time.sleep(random.uniform(2, 4))  
            break
        except TimeoutException:
            print(f"Page load timed out, retrying: {attempt + 1}")

    #==================================================================================================

    # Cookie consent - cookie all accept 
    cookie_selectors = ["sp-cc-accept", "a-button-text", "sp-cc-rejectall-button"]
    for selector in cookie_selectors:
        try:
            if selector == "sp-cc-accept":
                cookie_btn = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.ID, selector)))
            else:
                cookie_btn = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CLASS_NAME, selector)))
            cookie_btn.click()
            print("Cookie consent accepted")
            time.sleep(2)
            break
        except:
            continue

    #==================================================================================================

    print("Products are loading")
    list_data = []
    all_links = set()
    prev_count = 0
    scroll_count = 0
    max_scrolls = 100  
    no_new_products_count = 0

    #==================================================================================================

    # Məhsul linkləri üçün müxtəlif selectorlar
    product_selectors = [
        "a[data-testid='product-card-link']",
        "div.ProductCard-module__card_uyr_Jh7WpSkPx4iEpn4w a.a-link-normal",
        "a[data-testid='product-deal-card']",
        ".s-result-item a.a-link-normal",
        "a[data-csa-c-content-id]",
        ".dealContainer a",
        "[data-deal-id] a",
        ".a-cardui a[href*='/dp/']",
        "h2.a-size-mini a",
        "div[data-testid] a[href*='/dp/']",
        ".octopus-dlp-asin-title a",
        "div.ProductCard-module__imageWrapper_ytp7KTuzmF4mDTFCQLzr a"
    ]

    #==================================================================================================

    while scroll_count < max_scrolls:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(random.uniform(2, 4))
        
        # butun mehsul linkleri tapilir
        current_links = set()
        for selector in product_selectors:
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                links = [e.get_attribute("href") for e in elements 
                        if e.get_attribute("href") and "amazon.com" in e.get_attribute("href")]
                current_links.update(links)
            except:
                continue
        
    #==================================================================================================

        new_products = current_links - all_links
        if new_products:
            all_links.update(new_products)
            print(f"New product: {len(new_products)} | Count: {len(all_links)}")
            no_new_products_count = 0
        else:
            no_new_products_count += 1
        
        if no_new_products_count >= 8:
            print("New product is not found")
            break
        
        # Show more duymesi axtarilir
        load_more_selectors = [
            "a[aria-label='Load more results']",
            ".a-pagination .a-last a",
            "button[data-cy='load-more']",
            ".load-more-button",
            "a[data-testid='show-more-deals-link']",
            "button[data-testid='show-more-deals']",
            "#DealsGridScrollAnchor > div.DesktopDiscountAsinGrid-module__grid_pi4xEmM7RAHNMG9sGVBJ > div > div > div:nth-child(3) > div > button",
            "#DealsGridScrollAnchor button",
            "button[aria-label='See more deals']",
            "span[data-action='show-more-results']"

        ]
        
        for selector in load_more_selectors:
            try:
                load_more = driver.find_element(By.CSS_SELECTOR, selector)
                if load_more.is_displayed():
                    driver.execute_script("arguments[0].click();", load_more)
                    print("'Load more' button was clicked")
                    time.sleep(random.uniform(3, 5))
                    break
            except:
                continue
        
        scroll_count += 1

    print(f"Total number of products found: {len(all_links)}")

    #==================================================================================================

    seen_links = set()

    for i, link in enumerate(list(all_links)):
        if link in seen_links:
            continue
        seen_links.add(link)
        
        for attempt in range(2):
            try:
                driver.set_page_load_timeout(45)
                driver.get(link)
                time.sleep(random.uniform(1, 3))
                break
            except TimeoutException:
                print("Page load timeout")
                time.sleep(2)
                if attempt == 1:
                    continue

    #==================================================================================================

        # Səhifəni tam yükləməyi gözlə
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'h1')))
        except TimeoutException:
            print(f"The product page did not fully load: {link}")
            continue

    #==================================================================================================

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        data = {'URL': link}

    #==================================================================================================

        # Name
        name = soup.find("span", {"id": "productTitle"})
        data["Name"] = name.text.strip() if name else None

    #==================================================================================================

        # Star rate
        star_rate = soup.select_one("span.a-icon-alt")
        data["Star_Rating"] = star_rate.text.strip()[0] if star_rate else None

    #==================================================================================================

        # Ratings count
        pr_ratings = soup.find("span", {"id": "acrCustomerReviewText"})
        data["Product_Ratings"] = pr_ratings.text.strip() if pr_ratings else None

    #==================================================================================================

        # Breadcrumb
        breadcrumb_selectors = ["ul.a-unordered-list.a-horizontal.a-size-small", "#wayfinding-breadcrumbs_feature_div ul", ".a-breadcrumb ul"]
        categories = []
        for selector in breadcrumb_selectors:
            breadcrumb_ul = soup.select_one(selector)
            if breadcrumb_ul:
                for li in breadcrumb_ul.find_all("li"):
                    a_tag = li.find("a")
                    if a_tag:
                        categories.append(a_tag.get_text(strip=True))
                break
        data["Category"] = " › ".join(categories) if categories else None

    #==================================================================================================

        # Description
        description_selectors = ["div#productDescription", "div#feature-bullets ul", ".a-section.a-spacing-medium"]
        for selector in description_selectors:
            description_div = soup.select_one(selector)
            if description_div:
                data["Description"] = description_div.get_text(separator=" ", strip=True)
                break
        else:
            data["Description"] = None

    #==================================================================================================

        # Star rate and ratings
        reviews_list = []
        review_divs = soup.find_all("div", {"data-hook": "review-collapsed"})
        
        for div in review_divs:  
            text = div.get_text(separator=" ", strip=True)
            rating_tag = div.find_previous("i", {"data-hook": "review-star-rating"})
            rating = None
            if rating_tag:
                span = rating_tag.find("span", class_="a-icon-alt")
                if span:
                    try:
                        rating = float(span.get_text(strip=True).split()[0])  
                    except:
                        rating = None
            
            if text:
                reviews_list.append({"Review": text, "Rating": rating})
        
        data["Reviews"] = reviews_list

    #==================================================================================================

        list_data.append(data)
        print(f"A total of {len(list_data)} listings were collected")
        
    #==================================================================================================

        if len(list_data) % 10 == 0:
            save_data(list_data, "amazon_products.json")
            list_data = []


    #==================================================================================================

    if list_data:
        save_data(list_data, "amazon_products.json")

driver.quit()

Products are loading
New product: 78 | Count: 78
'Load more' button was clicked
'Load more' button was clicked
'Load more' button was clicked
New product: 24 | Count: 102
'Load more' button was clicked
New product: 36 | Count: 138
'Load more' button was clicked
New product: 12 | Count: 150
'Load more' button was clicked
New product: 64 | Count: 214
'Load more' button was clicked
'Load more' button was clicked
'Load more' button was clicked
'Load more' button was clicked
'Load more' button was clicked
New product: 127 | Count: 341
'Load more' button was clicked
'Load more' button was clicked
'Load more' button was clicked
'Load more' button was clicked
'Load more' button was clicked
'Load more' button was clicked
'Load more' button was clicked
'Load more' button was clicked
New product is not found
Total number of products found: 341
A total of 1 listings were collected
A total of 2 listings were collected
A total of 3 listings were collected
A total of 4 listings were collected
A total